In [ ]:
#I changed the names of the previous wrangled data (which was processed in a previous script)
#to be homogenous in excel, here I aggregate the whole dataset to create a unified panel dataset

import pandas as pd

panel2 = pd.read_csv('/Users/arthurwu/Desktop/final17.csv')
panel3 = pd.read_csv('/Users/arthurwu/Desktop/final18.csv')
panel4 = pd.read_csv('/Users/arthurwu/Desktop/final19.csv')

paneldata = pd.DataFrame()
paneldata = paneldata.append([panel2, panel3, panel4])
paneldata = paneldata.drop(columns = ['Unnamed: 0'])


In [ ]:
#standardizing the data
for i in range(2, paneldata.shape[1]):
    feature = paneldata.iloc[:, i]
    standardized = (feature - feature.mean())/feature.std()
    paneldata.iloc[:, i] = standardized
    
paneldata

In [ ]:
#the dependent variables come in the framework of one column section every year, I merged them here so it's just
#one column section overall.

def mergeDependents(path):
    output = pd.DataFrame()
    i = 1
    while True:
        try:
            index = i
            index = str(index)
            county = 'County.'+index
            deaths = 'Deaths.'+index
            pop = 'Population.'+index
            year = 'Year.'+index
            section = pd.read_excel(path, usecols = [county, deaths, pop, year])
            section.columns = ['County', 'Deaths', 'Population', 'Year']
            output = output.append(section)
            i = i+1
        except:
            break
    
    output['County'] = output['County'].str[:-11] + output['County'].str[-4:]
    output['County'] = output['County'].str.lower()
    output['depratio'] = output['Deaths']/output['Population']
    #implement lag here, where I lag everything by 1 year
    output['Year'] = output['Year'] - 1
    return output

In [ ]:
#conduct Panel regressions, one for each dependent variable
from linearmodels import PanelOLS
import statsmodels.api as sm
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

#paths for dependent variable data
paths = ['/Users/arthurwu/Desktop/circulatory.xlsx', '/Users/arthurwu/Desktop/external.xlsx', 
         '/Users/arthurwu/Desktop/mentalhealth.xlsx', '/Users/arthurwu/Desktop/nervous.xlsx',
        '/Users/arthurwu/Desktop/digestive.xlsx', '/Users/arthurwu/Desktop/neoplasms.xlsx', '/Users/arthurwu/Desktop/infectious.xlsx', 
         '/Users/arthurwu/Desktop/respiratory.xlsx', '/Users/arthurwu/Desktop/endocrine.xlsx']

output = pd.DataFrame({'Dependent Variable':[], 'Intercept':[], 'Beta':[], 'P-value':[], 'R^2':[], 'R^2 Overall':[], 'Degrees of Freedom':[],
                       '%Uninsured Beta':[], 'Unemployment Rate Beta':[], 'Per Capita Income Beta':[], "Bachelor's Degree Beta":[],
                      '%White Beta':[], '%Over 65 Beta':[], '%Below Poverty level Beta':[], 'Finance Value Beta':[]})

#iterate through each path to print a regression
for path in paths:
    df = mergeDependents(path)
    regdata = paneldata.merge(df, on=['County', 'Year'])
    regdata = regdata.set_index(['County', 'Year'])
    
    regdata = regdata.apply(pd.to_numeric, errors = 'coerce')
    endog = (regdata[['depratio']] - regdata[['depratio']].mean())/regdata[['depratio']].std()
    X = regdata[['perCapitaSpending', 'uninsured', 'unemployment', 'perCapitaIncome', 'bachelors', 'white', 'over65', 'poverty', 'financevalue']]
    exog = sm.add_constant(X)
    
    model = PanelOLS(endog, exog, entity_effects = True, time_effects = True, drop_absorbed = True)
    model = model.fit(use_lsdv = True, cov_type = 'clustered', cluster_entity = True, cluster_time = True)
    name = 'Proportion of Deaths due to ' + path.split('/')[-1].split('.')[0] + ' Disease'
    newrow = {'Dependent Variable': name, 'Intercept': model.params['const'], 'Beta': model.params['perCapitaSpending'], 
              'P-value': model.pvalues['perCapitaSpending'], 'R^2': model.rsquared, 
              'R^2 Overall': model.rsquared_overall, 'Degrees of Freedom': model.df_model, '%Uninsured Beta': model.params['uninsured'],
             'Unemployment Rate Beta': model.params['unemployment'], 'Per Capita Income Beta': model.params['perCapitaIncome'], 
             "Bachelor's Degree Beta": model.params['bachelors'], '%White Beta': model.params['white'], 
             '%Over 65 Beta': model.params['over65'], '%Below Poverty level Beta': model.params['poverty'], 
              'Finance Value Beta': model.params['financevalue'], 'Finance Value P-value': model.pvalues['financevalue']}
    output = output.append(newrow, ignore_index = True)

output = output.T
output.to_csv('regressions.csv')

In [ ]:
#get variance inflation factor

from statsmodels.stats.outliers_influence import variance_inflation_factor

#VIF for spending against all other variables
regdatax = regdata[['spending', 'death', 'inequality', 'density', 'uninsured', 'familysize', 'unemployment', 'income', 'bachelors', 'white', 'over65', 'poverty']]
regdatax = regdatax.dropna()
print('General VIF')
print(variance_inflation_factor(regdatax.values, 0))

#VIF for spending against death (particular VIF of concern)
regdatax2 = regdata[['spending', 'death']]
print('VIF vs General Deaths')
print(variance_inflation_factor(regdatax2.values, 0))


# Visualizing the Data with maps

In [ ]:
#import libraries
%matplotlib inline
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd

#read in relevant data and modify it so we can work with it
fipscodes = gpd.read_file('/Users/arthurwu/Desktop/UScounties/UScounties.shp')
fipscodes['location'] = fipscodes['NAME'].str.cat(fipscodes['STATE_NAME'], sep = ', ')
fipscodes['location'] = fipscodes['location'].str.upper()
fipscodes = fipscodes[['STATE_NAME', 'location', 'geometry', 'FIPS']]
fipscodes['location'] = fipscodes['location'].str.lower()

#read in health contracts counties and truncate data
healthcontracts = pd.read_csv("/Users/arthurwu/Desktop/actionobligation2019outliersincluded.csv")
percentiles(healthcontracts)
healthcontracts = healthcontracts.loc[(healthcontracts['percentile'] > 1) & (healthcontracts['percentile'] < 99)]
healthcontracts = healthcontracts.merge(covars, on='location')
healthcontracts['perCapitaSpending'] = healthcontracts['federal_action_obligation']/healthcontracts['pop2017']
healthcontracts['location'] = healthcontracts['location'].str.lower()
thedata = fipscodes.merge(healthcontracts, on = 'location')
print(thedata.shape)

#read in US counties shpfile to serve as the base
usacounties = gpd.read_file('/Users/arthurwu/Desktop/UScounties/UScounties.shp')

#plot a heatmap of US counties based on their contracts income
fig, plane = plt.subplots(figsize = (20, 20))
usacounties.plot(ax = plane, color = 'grey', edgecolor = 'black')
thedata.plot(ax = plane, column = 'perCapitaSpending', legend = True, 
             legend_kwds = {'label':"Health Contract Value Per Capita 2019", 'orientation':"horizontal"})

